<a href="https://colab.research.google.com/github/adigenova/uohdb/blob/main/code/SQL_V_chinook_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with the Chinook database

## downloading the database from github

In [1]:
!pip install SQLAlchemy==1.4.46
#download the Chinook database
!git clone https://github.com/lerocha/chinook-database.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Cloning into 'chinook-database'...
remote: Enumerating objects: 1960, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1960 (delta 33), reused 50 (delta 21), pack-reused 1885
Receiving objects: 100% (1960/1960), 19.57 MiB | 17.14 MiB/s, done.
Resolving deltas: 100% (1277/1277), done.


In [3]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql
# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite


### Display tables of the Chinook database


In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


name
Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
MediaType
Playlist
PlaylistTrack


getting information about a particular table


In [ ]:
%%sql 
PRAGMA table_info([Artist]);

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,ArtistId,INTEGER,1,None,1
1,Name,NVARCHAR(120),0,None,0


getting the SQL code that generate a table


## Create automatic documentation with [schemaspy](https://schemaspy.org/)

<img src="https://schemaspy.org/sample/diagrams/summary/relationships.real.large.png" alt="ER Diagram" width="836" height="605" />


In the above ER diagram, the tiny vertical key icon indicates a column is a primary key. A primary key is a column (or set of columns) whose values uniquely identify every row in a table. For example, `Employeeud` is the primary key in the `Employee` table.

The relationship icon indicates a foreign key constraint and a one-to-many relationship. 




# SQL con Chinook





## Consultas Avanzadas

In [ ]:
# cuales son los 5 artistas que venden mas canciones?
%%sql
SELECT a.Name 'Artista', sum(li.UnitPrice) 'Total vendido' 
FROM InvoiceLine li, Track t, Album al, Artist a
WHERE li.TrackId = t.Trackid 
	and al.AlbumId = t.AlbumId 
	and a.ArtistId = al.ArtistId
GROUP BY a.Name
ORDER BY COUNT(a.ArtistId) DESC
limit 5;

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Artista,Total vendido
Iron Maiden,138.5999999999998
U2,105.92999999999982
Metallica,90.0899999999999
Led Zeppelin,86.12999999999992
Os Paralamas Do Sucesso,44.550000000000004


In [ ]:
#INNER Join
#select count(*) from album JOIN Artist ON album.artistId=Artist.artistId limit 5;
%%sql
SELECT a.Name 'Artista', sum(li.UnitPrice) 'Total vendido' 
FROM InvoiceLine as li join Track USING (TrackId) join Album USING (AlbumId) join Artist as a USING (ArtistId)
GROUP BY a.Name
ORDER BY COUNT(a.ArtistId) DESC
limit 5;

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Artista,Total vendido
Iron Maiden,138.5999999999998
U2,105.92999999999982
Metallica,90.0899999999999
Led Zeppelin,86.12999999999992
Os Paralamas Do Sucesso,44.550000000000004


In [ ]:
# Cuales son las canciones más compradas del año 2012?
%%sql
select t.name, count(t.trackid) purchaseCount
from track t 
join invoiceline l on l.trackid = t.trackid
join invoice i on l.invoiceid = i.invoiceid
where  i.invoicedate between datetime("2012-01-01") and datetime("2012-12-31")
group by t.name
order by purchaseCount DESC
limit 5

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Name,purchaseCount
Black Dog,2
Dream Of Mirrors,2
Iron Maiden,2
Polly,2
Powerslave,2


In [ ]:
# Cuales son los tipos de medio más vendido?
%%sql
SELECT m.Name AS 'Tipo de Medio', COUNT(m.MediaTypeId) 'Total' 
FROM InvoiceLine li, Track t, MediaType m
WHERE li.TrackId = t.TrackId
	and m.MediaTypeId = t.MediaTypeId
GROUP BY m.Name
ORDER BY m.MediaTypeId;

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Tipo de Medio,Total
MPEG audio file,1976
Protected AAC audio file,146
Protected MPEG-4 video file,111
Purchased AAC audio file,4
AAC audio file,3


In [ ]:
# mejor vendedor del 2009
%%sql
select e.firstName || " " || e.lastName as 'Mejor Vendedor 2009', sum(i.total) as Total 
from Invoice i
join customer c on c.customerid =  i.customerid
join employee e on e.employeeid = c.supportrepid
where i.invoiceDate like '2009%'
group by e.Employeeid
order by sum(i.total) desc
limit 1

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Mejor Vendedor 2009,Total
Steve Johnson,164.33999999999997


In [ ]:
#mejor venededor del 2019
%%sql
select "Mejor Vendedor 2009", max("Total") from (
select e.firstName || " " || e.lastName as "Mejor Vendedor 2009", sum(i.total) as "Total" from Invoice i
join customer c on c.customerid =  i.customerid
join employee e on e.employeeid = c.supportrepid
where i.invoiceDate like '2009%'
group by e.Employeeid
)

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Mejor Vendedor 2009,"max(""Total"")"
Steve Johnson,164.33999999999997


In [ ]:
# Muestre todoas las canciones pero sin el ID. El resultado debe incluir el nommbre del Album, el typo de Medio y el Genero
%%sql
select t.name as 'Nombre Cancion', t.composer, t.unitprice, 
a.title as 'Nombre Album', m.name as 'Tipo de Medio',  g.name as 'Nombre de Genero' from Track t
join album a on a.albumid = t.albumid
join genre g on g.genreid = t.genreid
join MediaType m on m.mediatypeid = t.mediatypeid
limit 10

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Nombre Cancion,Composer,UnitPrice,Nombre Album,Tipo de Medio,Nombre de Genero
For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock
Balls to the Wall,None,0.99,Balls to the Wall,Protected AAC audio file,Rock
Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99,Restless and Wild,Protected AAC audio file,Rock
Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99,Restless and Wild,Protected AAC audio file,Rock
Princess of the Dawn,Deaffy & R.A. Smith-Diesel,0.99,Restless and Wild,Protected AAC audio file,Rock
Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock
Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock
Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock
Snowballed,"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock
Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",0.99,For Those About To Rock We Salute You,MPEG audio file,Rock


In [ ]:
#construya una consulta que muestre los clientes de que no son de USA
%%sql
select c.FirstName || " " || c.LastName 'Nombre Cliente', c.Country 'Pais'
from Customer c
where c.Country != "USA"
limit 10

 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


Nombre Cliente,Pais
Luís Gonçalves,Brazil
Leonie Köhler,Germany
François Tremblay,Canada
Bjørn Hansen,Norway
František Wichterlová,Czech Republic
Helena Holý,Czech Republic
Astrid Gruber,Austria
Daan Peeters,Belgium
Kara Nielsen,Denmark
Eduardo Martins,Brazil


In [4]:
%%sql
SELECT c.FirstName || ' ' || c.LastName AS CustomerName,
       (SELECT COUNT(*) FROM invoiceline WHERE InvoiceId IN
           (SELECT InvoiceId FROM invoice WHERE CustomerId = c.CustomerId)) AS TotalTracksPurchased,
       (SELECT COUNT(DISTINCT AlbumId) FROM
           (SELECT AlbumId FROM invoiceline INNER JOIN track ON invoiceline.TrackId = track.TrackId
            WHERE InvoiceId IN (SELECT InvoiceId FROM invoice WHERE CustomerId = c.CustomerId))) AS TotalAlbumsPurchased,
       (SELECT SUM(Total) FROM invoice WHERE CustomerId = c.CustomerId) AS TotalSpent
FROM customer c;


 * sqlite:////content/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Done.


CustomerName,TotalTracksPurchased,TotalAlbumsPurchased,TotalSpent
Luís Gonçalves,38,22,39.62
Leonie Köhler,38,22,37.620000000000005
François Tremblay,38,26,39.62
Bjørn Hansen,38,25,39.62
František Wichterlová,38,22,40.620000000000005
Helena Holý,38,19,49.620000000000005
Astrid Gruber,38,21,42.62
Daan Peeters,38,19,37.62
Kara Nielsen,38,20,37.620000000000005
Eduardo Martins,38,25,37.620000000000005
